# Create weights for OM4p5

In [1]:
import xarray as xr
import xesmf
import numpy as np

In [2]:
eradir = '/local2/home/dev/dev_forcings/precip_erainterim_correction_gpcp/'

## Building the grids

In [3]:
# ERA-interim
erai_grid = xr.open_dataset(f'{eradir}/erai_grid.nc')

lon = erai_grid['lon'].values
lon_bnds = np.concatenate((np.array([lon[0] -0.5 * 0.7031]), 0.5 * (lon[:-1] + lon[1:]), np.array([lon[-1] + 0.5 * 0.7031])), axis=0)

lat = erai_grid['lat'].values
lat_bnds = np.concatenate((np.array([-90]), 0.5 * (lat[:-1] + lat[1:]), np.array([90])), axis=0)

erai_grid['lon_b'] = xr.DataArray(data=lon_bnds, dims=('lonp1'))
erai_grid['lat_b'] = xr.DataArray(data=lat_bnds, dims=('latp1'))

In [4]:
# OM4p5
om4dir='/archive/gold/datasets/OM4_05/mosaic_ocean.v20180227.unpacked'
om4_supergrid = xr.open_dataset(f'{om4dir}/ocean_hgrid.nc')

In [5]:
om4_supergrid

<xarray.Dataset>
Dimensions:   (nx: 1440, nxp: 1441, ny: 1152, nyp: 1153)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    tile      |S255 ...
    y         (nyp, nxp) float64 ...
    x         (nyp, nxp) float64 ...
    dy        (ny, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    area      (ny, nx) float64 ...
    angle_dx  (nyp, nxp) float64 ...

In [6]:
om4_grid = xr.Dataset()
lon = om4_supergrid['x'].values[1::2,1::2].copy()
lat = om4_supergrid['y'].values[1::2,1::2].copy()
lon_b = om4_supergrid['x'].values[0::2,0::2].copy()
lat_b = om4_supergrid['y'].values[0::2,0::2].copy()

om4_grid['lon'] = xr.DataArray(data=lon, dims=('y', 'x'))
om4_grid['lat'] = xr.DataArray(data=lat, dims=('y', 'x'))
om4_grid['lon_b'] = xr.DataArray(data=lon_b, dims=('yp1', 'xp1'))
om4_grid['lat_b'] = xr.DataArray(data=lat_b, dims=('yp1', 'xp1'))

In [7]:
om4_grid

<xarray.Dataset>
Dimensions:  (x: 720, xp1: 721, y: 576, yp1: 577)
Dimensions without coordinates: x, xp1, y, yp1
Data variables:
    lon      (y, x) float64 -299.8 -299.2 -298.8 -298.2 ... 59.99 59.99 60.0
    lat      (y, x) float64 -77.91 -77.91 -77.91 -77.91 ... 65.39 65.18 64.97
    lon_b    (yp1, xp1) float64 -300.0 -299.5 -299.0 -298.5 ... 60.0 60.0 60.0
    lat_b    (yp1, xp1) float64 -78.0 -78.0 -78.0 -78.0 ... 65.29 65.08 64.87

In [8]:
%time regrid_conserve = xesmf.Regridder(erai_grid, om4_grid, method='conservative', periodic=True)

Create weight file: conservative_256x512_576x720.nc
CPU times: user 16.8 s, sys: 650 ms, total: 17.4 s
Wall time: 17.6 s


In [9]:
%time regrid_patch = xesmf.Regridder(erai_grid, om4_grid, method='patch', periodic=True)

Create weight file: patch_256x512_576x720_peri.nc
CPU times: user 1min 12s, sys: 2.48 s, total: 1min 14s
Wall time: 1min 14s


In [10]:
%time regrid_bilin = xesmf.Regridder(erai_grid, om4_grid, method='bilinear', periodic=True)

Create weight file: bilinear_256x512_576x720_peri.nc
CPU times: user 8.95 s, sys: 609 ms, total: 9.56 s
Wall time: 9.55 s
